# How SVMs Make Decisions: Margins and Support Vectors

Support Vector Machines (SVMs) make classification decisions by finding the hyperplane that maximally separates data points of different classes. This separation is defined by the "margin," which is the distance between the hyperplane and the nearest data points from each class. These closest points are called "support vectors," and they are the critical elements that determine the position and orientation of the optimal hyperplane and the size of the margin.

##Setup, Imports, Utility Functions

In [6]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plta
from sklearn.datasets import make_classification
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from matplotlib.colors import ListedColormap
import ipywidgets as widgets
from ipywidgets import interact, fixed
from IPython.display import display, clear_output

In [7]:
def generate_data(n_samples=100, n_features=2, class_sep=1.0, random_state=42):
    """
    Generate a simple classification dataset for SVM visualization

    Parameters:
    -----------
    n_samples: int, number of samples to generate
    n_features: int, number of features (keep at 2 for visualization)
    class_sep: float, separation between classes
    random_state: int, seed for reproducibility

    Returns:
    --------
    X: feature matrix
    y: target vector
    """
    X, y = make_classification(
        n_samples=n_samples,
        n_features=n_features,
        n_informative=2,
        n_redundant=0,
        n_clusters_per_class=1,
        class_sep=class_sep,
        random_state=random_state
    )
    return X, y

In [8]:
def plot_decision_boundary(model, X, y, show_support_vectors=True, ax=None):
    """
    Plot the decision boundary and the support vectors of an SVM model

    Parameters:
    -----------
    model: trained SVM model
    X: feature matrix
    y: target vector
    show_support_vectors: bool, whether to highlight support vectors
    ax: matplotlib axes object

    Returns:
    --------
    ax: matplotlib axes object with the plot
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 8))

    # Define the mesh grid to plot the decision boundary
    h = .02  # step size in the mesh
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    # Predict on the mesh grid
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # Create a custom colormap for the decision regions
    cmap = ListedColormap(['#FFAAAA', '#AAAAFF'])

    # Plot the decision boundary
    ax.contourf(xx, yy, Z, alpha=0.8, cmap=cmap)

    # Highlight the margin if the kernel is linear
    if model.kernel == 'linear' and hasattr(model, 'coef_'):
        # Get the coefficients of the hyperplane
        w = model.coef_[0]
        a = -w[0] / w[1]
        xx = np.linspace(x_min, x_max)

        # Decision boundary
        yy = a * xx - (model.intercept_[0]) / w[1]
        ax.plot(xx, yy, 'k-', label='Decision boundary')

        # Margins
        margin = 1 / np.sqrt(np.sum(model.coef_ ** 2))
        yy_down = yy - np.sqrt(1 + a ** 2) * margin
        yy_up = yy + np.sqrt(1 + a ** 2) * margin
        ax.plot(xx, yy_down, 'k--', label='Margin')
        ax.plot(xx, yy_up, 'k--')

    # Plot the data points
    scatter = ax.scatter(X[:, 0], X[:, 1], c=y, cmap=ListedColormap(['red', 'blue']),
                       edgecolors='k', marker='o', s=80, alpha=0.7)

    # Highlight support vectors
    if show_support_vectors and hasattr(model, 'support_vectors_'):
        ax.scatter(model.support_vectors_[:, 0], model.support_vectors_[:, 1],
                 s=200, facecolors='none', edgecolors='green', linewidths=2,
                 marker='o', label='Support Vectors')

    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xlabel('Feature 1')
    ax.set_ylabel('Feature 2')
    ax.legend()

    return ax

## Step 1: Understanding Margins in SVMs

In [9]:
# Create a widget for generating data with varying separation
@widgets.interact(class_separation=(0.5, 4.0, 0.1), n_samples=(50, 500, 10))
def visualize_data(class_separation=1.0, n_samples=100):
    X, y = generate_data(n_samples=n_samples, class_sep=class_separation)

    fig, ax = plt.subplots(figsize=(10, 8))
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap=ListedColormap(['red', 'blue']),
               edgecolors='k', marker='o', s=80)

    ax.set_title("Dataset with Two Classes")
    ax.set_xlabel("Feature 1")
    ax.set_ylabel("Feature 2")

    plt.show()

    return X, y

interactive(children=(FloatSlider(value=1.0, description='class_separation', max=4.0, min=0.5), IntSlider(valu…

##Step 2: Introduction to Support Vectors

In [10]:
# This section will demonstrate the concept of support vectors
# #added during screencast
def illustrate_support_vectors():
    # Generate a simple dataset
    X, y = generate_data(n_samples=50, class_sep=1.0, random_state=42)

    # Train a linear SVM
    svm_model = SVC(kernel='linear', C=1.0)
    svm_model.fit(X, y)

    # Plot with highlighted support vectors
    fig, ax = plt.subplots(figsize=(10, 8))
    plot_decision_boundary(svm_model, X, y, show_support_vectors=True, ax=ax)
    ax.set_title("Support Vectors Highlighted in Green")
    plt.show()

    print(f"Number of support vectors: {len(svm_model.support_vectors_)}")
    print("These points are crucial in defining the decision boundary and margin")

illustrate_support_vectors()


NameError: name 'plt' is not defined

##Step 3: Training the SVM with Scikit-learn

In [11]:
# Train an SVM model on our dataset
# #added during screencast
def train_svm(X, y, kernel='linear', C=1.0):
    """
    Train an SVM model and return it

    Parameters:
    -----------
    X: feature matrix
    y: target vector
    kernel: string, kernel type
    C: float, regularization parameter

    Returns:
    --------
    svm_model: trained SVM model
    """
    svm_model = SVC(kernel=kernel, C=C)
    svm_model.fit(X, y)

    print(f"Model trained with {kernel} kernel and C={C}")
    print(f"Number of support vectors: {len(svm_model.support_vectors_)}")

    return svm_model

# Create interactive widgets for model training
def interactive_svm_training():
    X, y = generate_data(n_samples=100)

    kernel_selector = widgets.Dropdown(
        options=['linear', 'rbf', 'poly'],
        value='linear',
        description='Kernel:',
    )

    C_slider = widgets.FloatLogSlider(
        value=1.0,
        base=10,
        min=-2, # 10^-2
        max=2,  # 10^2
        step=0.1,
        description='C:',
    )

    output = widgets.Output()

    def on_change(change):
        with output:
            clear_output()
            model = train_svm(X, y, kernel=kernel_selector.value, C=C_slider.value)
            fig, ax = plt.subplots(figsize=(10, 8))
            plot_decision_boundary(model, X, y, ax=ax)
            plt.show()

    kernel_selector.observe(on_change, names='value')
    C_slider.observe(on_change, names='value')

    # Initial display
    with output:
        model = train_svm(X, y, kernel=kernel_selector.value, C=C_slider.value)
        fig, ax = plt.subplots(figsize=(10, 8))
        plot_decision_boundary(model, X, y, ax=ax)
        plt.show()

    return widgets.VBox([widgets.HBox([kernel_selector, C_slider]), output])

# Button to start interactive SVM training
button_train = widgets.Button(
    description="Train SVM Model",
    button_style="success",
    tooltip="Click to start interactive SVM training"
)

def on_button_train_clicked(b):
    display(interactive_svm_training())

button_train.on_click(on_button_train_clicked)
display(button_train)

Button(button_style='success', description='Train SVM Model', style=ButtonStyle(), tooltip='Click to start int…

##Step 4: Exploring the Impact of Margins and Support Vectors

In [ ]:
# Create an interactive function to compare different C values
def compare_margins():
    # Generate data
    X, y = generate_data(n_samples=100, class_sep=1.0)

    # Create C value slider
    c_values = widgets.FloatLogSlider(
        value=1.0,
        base=10,
        min=-3, # 10^-3
        max=3,  # 10^3
        step=0.1,
        description='C value:',
        continuous_update=False
    )

    # Create toggle for support vectors
    sv_toggle = widgets.Checkbox(
        value=True,
        description='Show support vectors',
    )

    output = widgets.Output()

    def update_plot(c=1.0, show_sv=True):
        with output:
            clear_output(wait=True)

            # Train models with different C values
            model_current = SVC(kernel='linear', C=c)
            model_current.fit(X, y)

            # Plot the results
            fig, ax = plt.subplots(figsize=(12, 10))
            plot_decision_boundary(model_current, X, y, show_support_vectors=show_sv, ax=ax)
            ax.set_title(f'SVM with C={c:.3f} ({len(model_current.support_vectors_)} support vectors)')
            plt.show()

            # Print explanations
            if c < 0.1:
                print("LOOSE MARGIN (small C value):")
                print("- Creates a wider margin")
                print("- More points may be inside or on the wrong side of the margin")
                print("- More support vectors used")
                print("- Less sensitive to individual points, better generalization for noisy data")
            elif c > 10:
                print("STRICT MARGIN (large C value):")
                print("- Creates a narrower margin")
                print("- Fewer points inside or on the wrong side of the margin")
                print("- Fewer support vectors used")
                print("- More sensitive to individual points, may overfit")
            else:
                print("BALANCED MARGIN (moderate C value):")
                print("- Balances between margin width and classification errors")
                print("- Moderate number of support vectors")

            print(f"\nNumber of support vectors: {len(model_current.support_vectors_)} out of {len(X)} points")

    # Connect the widgets
    interact_manual = widgets.interactive_output(
        update_plot,
        {'c': c_values, 'show_sv': sv_toggle}
    )

    # Display the initial plot
    with output:
        update_plot()

    # Arrange the widgets
    controls = widgets.VBox([c_values, sv_toggle])
    ui = widgets.HBox([controls, output])

    return ui

# Create button to start the margin comparison
button_compare = widgets.Button(
    description="Compare Margins",
    button_style="primary",
    tooltip="Click to explore SVM margins with different C values"
)

def on_button_compare_clicked(b):
    display(compare_margins())

button_compare.on_click(on_button_compare_clicked)
display(button_compare)